In [1]:
import numpy as np
from numpy import asarray
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from sklearn.decomposition import PCA
import os
from os import path
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#define RMSE as a function, since we'll use this in the NN model 
def rmse(target,prediction):
    return(np.sqrt(((target - prediction)**2).sum()/len(target)))

cwd = os.getcwd()
cwd

import joblib

In [2]:
### PREP DATA ###

## MANUAL INPUT ## -- locate file
sensor = 'PRISMA' #'PRISMA', 'WV3', 'Landsat8', 'terraspec'
target_file = 'C:/Users/htccr/Documents/Aconquija/python_scripts/outputs/' + sensor + '_spots.xlsx'
sensor_meta = pd.read_csv('C:/Users/htccr/Documents/Aconquija/python_scripts/keep/'+ sensor + '_meta.csv', header = 0, index_col = None)
 
data = pd.read_excel(target_file, sheet_name = 'band_mean', header = None, index_col = None) # choose sheet: 'band_mean', 'continuum', 'continuum_removed'
data_std = pd.read_excel(target_file, sheet_name = 'band_std', header = None, index_col = None)
data_vars = pd.read_excel(target_file, sheet_name = 'variables', header = 0, index_col = None)
spots_meta = pd.read_excel('C:/Users/htccr/Documents/Aconquija/python_scripts/outputs/spots_meta.xlsx',sheet_name = 'Sheet1', header = 0, index_col = None)

# decompact data_vars
age = np.array(data_vars['age'])
age_sd = np.array(data_vars['age_sd'])
age_n = np.array(data_vars['age_n'])
fan = np.array(data_vars['fan'])
unit = np.array(data_vars['unit'])
unique = np.array(data_vars['unique'])
pix_count = np.array(data_vars['pix_count'])

# decompact sensor metadata (e.g. nm, band names) and create dictionaries
band_names = sensor_meta.Band
nm_names = sensor_meta.nm.values
nm_names = np.round(nm_names,1)
band_dict_nm = pd.Series(sensor_meta.Name.values,index=nm_names).to_dict()
band_dict_idx  = pd.Series(sensor_meta.Name.values,index=sensor_meta.index.values).to_dict() # create dictionary of band names to wavelength
nm_dict = pd.Series(sensor_meta.nm.values,index=sensor_meta.Band).to_dict() # create dictionary of band names to wavelength

if 'bad_age_idx' in locals():
    del bad_age_idx
if 'bad_band_idx' in locals():
    del bad_band_idx

## MANUAL INPUT ## -- set bands and/or ages to drop
bad_band_idx = np.r_[97:110,141:169,226:230] # choose bands to drop by index
bad_age_idx = np.where((pix_count == 0) | (pix_count <10))# (age == 3.37) | (age == 16.48) | (age == 78.5) | (age == 314.48) )# | (unit == 'Q2.5a') | (unit == 'Q2.5b')) #  choose age to drop (two decimels)
if 'bad_age_idx' in locals():
    data = data.drop(np.r_[bad_age_idx], axis=0)
    data_std = data_std.drop(np.r_[bad_age_idx], axis=0)
    age = np.delete(age, bad_age_idx)
    age_sd = np.delete(age_sd, bad_age_idx)
    age_n = np.delete(age_n, bad_age_idx)
    fan = np.delete(fan, bad_age_idx)
    unit = np.delete(unit, bad_age_idx)
    unique = np.delete(unique, bad_age_idx)
    pix_count = np.delete(pix_count, bad_age_idx)
    spots_meta = spots_meta.drop(np.r_[bad_age_idx], axis=0)
    
    data_plot = data.copy()
    data_std_plot = data_std.copy()
    print('some ages dropped')
else:
    data_plot = data.copy()
    data_std_plot = data_std.copy()
    print('No ages dropped')
    
# drop chosen values from dataset(s)
if 'bad_band_idx' in locals():
    #data = data.drop(np.r_[bad_band_idx], axis=1)
    #nm_names = np.delete(nm_names, bad_band_idx)
    data.iloc[:,bad_band_idx] = np.nan # if bad bands chosen, set band bands as 0 for PCA` #!!change to 0 if NOT normalizign first!!!
    data_std.iloc[:,bad_band_idx] = np.nan #!!change to 0 if NOT normalizign first!!!
    
    # data for plotting (nan instead of 0 for bad bands)
    data_plot = data.copy()
    data_std_plot = data_std.copy()
    data_plot.iloc[:,bad_band_idx] = np.nan
    data_std_plot.iloc[:,bad_band_idx] = np.nan
    print('some bands dropped')
else:
    # data for plotting (nan instead of 0 for bad bands)
    data_plot = data.copy()
    data_std_plot = data_std.copy()
    print('No bands dropped')

print(np.shape(data))

some ages dropped
some bands dropped
(20, 230)


In [3]:
# import model data
# x_data
x_dat = pd.DataFrame(data)
x_dat = x_dat.reset_index(drop=True)
x_dat = x_dat.dropna(axis='columns').reset_index(drop=True)

scaler_x = StandardScaler()
x_dat_norm = scaler_x.fit_transform(x_dat)
print(x_dat.head())

# y data
y_dat = pd.DataFrame({'age':age})
print(y_dat.head())

        0         1         2         3         4         5         6    \
0  0.095016  0.087031  0.086043  0.092358  0.097595  0.104654  0.110789   
1  0.072552  0.066371  0.066350  0.070230  0.074420  0.080428  0.086002   
2  0.092628  0.084281  0.082825  0.088585  0.095247  0.103068  0.108866   
3  0.076524  0.071851  0.070685  0.075591  0.079310  0.085356  0.090487   
4  0.077176  0.071142  0.070311  0.074786  0.078883  0.084782  0.090278   

        7         8         9    ...       216       217       218       219  \
0  0.113885  0.115541  0.120763  ...  0.223382  0.205818  0.205955  0.233426   
1  0.088019  0.089246  0.093221  ...  0.178489  0.164673  0.162031  0.184269   
2  0.111967  0.113537  0.119765  ...  0.231434  0.212615  0.216693  0.238236   
3  0.093323  0.095023  0.099331  ...  0.208660  0.193258  0.187744  0.215425   
4  0.092662  0.094946  0.099416  ...  0.206747  0.190891  0.185394  0.210658   

        220       221       222       223       224       225  
0  0

In [4]:
#import all required libraries etc

from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm
import itertools

#run notebook with functions we'll need
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
#         print(excluded)
        new_pval = pd.Series(index=excluded, dtype = 'float64')
        for new_column in excluded:
#             print(included, new_column)
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = excluded[new_pval.argmin()]
#             print(best_feature, included)
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(X.columns[worst_feature], worst_pval))
        if not changed:
            break
    return included

### Run MLR on training data

In [5]:
# organize data for MLP nueral network model
xy = x_dat
xy['age'] = y_dat

xy = xy.sample(frac=1).reset_index(drop=True)
xy = xy.sample(frac=1).reset_index(drop=True) # shuffle again

x = xy.drop(labels = 'age', axis=1)
y = xy['age']

In [6]:
# SEQUENTIAL SELECTION OF TRAIN/TEST & STANDARDIZED

fracTrain = .95 #fraction of data to use for training
ntrain = int(len(y)*fracTrain)

x_train = x[:ntrain] #train on n observations
y_train = y[:ntrain]

x_test = x[ntrain:] #test on remaining observations
y_test = y[ntrain:]

print(np.shape(x_train))
print(np.shape(x_test))

(19, 185)
(1, 185)


In [ ]:
lm_MLR = linear_model.LinearRegression()
model = lm_MLR.fit(x_train,y_train)
ypred_train = lm_MLR.predict(x_train) #y predicted by MLR
ypred_test = lm_MLR.predict(x_test) #y predicted by MLR
intercept_MLR = lm_MLR.intercept_ #intercept predicted by MLR
coef_MLR = lm_MLR.coef_ #regression coefficients in MLR model
R2_train= lm_MLR.score(x_train, y_train) #R-squared value from MLR model
R2_test = lm_MLR.score(x_test, y_test) #R-squared value from MLR model


print('MLR results:')
print('a0 = ' + str(intercept_MLR))
print('a1 = ' + str(coef_MLR[0]))
print('a2 = ' + str(coef_MLR[1]))
print('a3 = ' + str(coef_MLR[2]))
print('a4 = ' + str(coef_MLR[3]))
#print('a5 = ' + str(coef_MLR[4]))
#print('a6 = ' + str(coef_MLR[5]))
#print('etc...')
R2_train

In [ ]:
plt.figure()
plt.scatter(y_train, ypred_train)
plt.xlab = ('obs')
plt.ylab = ('predicted')
plt.title = (R2_MLR)
R2_train

In [ ]:
plt.figure()
plt.scatter(y_test, ypred_test)
plt.xlab = ('obs')
plt.ylab = ('predicted')
plt.title = (R2_MLR)

R2_test

In [13]:
x_train = x_train.T.reset_index(drop=True).T
y_train = y_train.T.reset_index(drop=True).T

In [14]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,175,176,177,178,179,180,181,182,183,184
0,0.072552,0.066371,0.066350,0.070230,0.074420,0.080428,0.086002,0.088019,0.089246,0.093221,...,0.178489,0.164673,0.162031,0.184269,0.160523,0.173540,0.132066,0.157797,0.173261,0.163852
1,0.079577,0.073320,0.072249,0.077755,0.081926,0.088212,0.093866,0.096745,0.098245,0.102803,...,0.223618,0.205754,0.200732,0.229513,0.196232,0.213340,0.164392,0.197503,0.219205,0.206597
2,0.082368,0.077101,0.076679,0.082336,0.084593,0.090949,0.097175,0.100718,0.102927,0.107706,...,0.233048,0.217796,0.208759,0.238459,0.208309,0.226521,0.178408,0.201354,0.232492,0.217712
3,0.077450,0.069985,0.069072,0.072671,0.077832,0.083862,0.088644,0.091139,0.092495,0.096264,...,0.189033,0.173987,0.174992,0.196828,0.168978,0.183795,0.139680,0.173438,0.185772,0.176120
4,0.077176,0.071142,0.070311,0.074786,0.078883,0.084782,0.090278,0.092662,0.094946,0.099416,...,0.206747,0.190891,0.185394,0.210658,0.182139,0.199880,0.152988,0.179091,0.202842,0.190318
5,0.070150,0.059038,0.056890,0.060561,0.067528,0.073183,0.078201,0.079958,0.080595,0.085106,...,0.195606,0.178975,0.191459,0.204127,0.178786,0.188211,0.143359,0.193871,0.187603,0.190170
6,0.085625,0.079147,0.079584,0.084716,0.089385,0.095279,0.102000,0.105042,0.107614,0.112889,...,0.238815,0.221087,0.218034,0.245393,0.211069,0.231297,0.181030,0.211560,0.237286,0.223895
7,0.083217,0.078435,0.077607,0.083004,0.090908,0.098830,0.104415,0.107677,0.110113,0.115633,...,0.223342,0.203991,0.213451,0.232009,0.199888,0.215206,0.161747,0.212239,0.210820,0.216187
8,0.063855,0.058381,0.056903,0.061869,0.065720,0.071742,0.076783,0.079598,0.081136,0.085483,...,0.203395,0.188700,0.185532,0.209826,0.182502,0.197081,0.152599,0.182594,0.198907,0.190579
9,0.092628,0.084281,0.082825,0.088585,0.095247,0.103068,0.108866,0.111967,0.113537,0.119765,...,0.231434,0.212615,0.216693,0.238236,0.205556,0.223030,0.167886,0.215340,0.225796,0.224041


In [ ]:
np.shape(y_train)

In [22]:
#now, use stepwise regression to find which predictors to use
import warnings
warnings.filterwarnings('ignore')
result = stepwise_selection(x_train, y_train, threshold_in=.1, threshold_out=0.01)
print('resulting features:')
print(result)

Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.0686812
Drop                            181 with p-value 0.0686812
Add                             181 with p-value 0.06868

KeyboardInterrupt: 

In [ ]:
# insert 0 coefficients at all nm values that are 'bad bands'
coef_MLR_all = list(coef_MLR)

for idx in bad_band_idx:
    coef_MLR_all.insert(idx,0)

In [ ]:
plt.plot(nm_names, coef_MLR_all)

In [ ]:
nm_names[np.where(coef_MLR_all == np.min(coef_MLR_all))]